In [1]:

import pandas as pd
import pylab as plt
import seaborn
from sklearn.linear_model import LinearRegression
import numpy.random as nprnd
import random
import json
import numpy as np
from sklearn.model_selection import train_test_split
from scipy.spatial.distance import cosine
from sklearn.metrics import pairwise_distances
from scipy.sparse import coo_matrix
from sklearn.decomposition import LatentDirichletAllocation
import lda

pd.set_option('display.max_columns', 500)
%matplotlib inline

/home/ubuntu/anaconda3/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


# Data Gathering and Preparation

## Data prepocessing and Data integrity checks

In [2]:
# Read data
df = pd.read_csv("music.tsv",delimiter='\t',encoding='utf-8',header=None)
# Rename the columns
df.columns = ['userID','playerID','playerName','playNum']
# Remove the missing values
df.dropna(inplace=True)
# Data cleaning and turned the complicated ID format to the simple one.
_, user_id = np.unique(df.userID, return_inverse=True)
_, player_id = np.unique(df.playerID, return_inverse=True)
df['userID'] = user_id
df['playerID'] = player_id
df['userID'] = df['userID'].astype(np.int32)
df['playerID'] = df['playerID'].astype(np.int32)
df['playNum'] = df['playNum'].astype(np.int32)

s_player = df[['playerID', 'playerName']].drop_duplicates(subset='playerID').set_index('playerID').sort_index()
df.drop('playerName', axis=1, inplace=True)
df.head()

,userID,playerID,playNum
0,0,37425,2137
1,0,152038,1099
2,0,112364,897
3,0,38434,717
4,0,117441,706


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17309316 entries, 0 to 17535654
Data columns (total 3 columns):
userID      int32
playerID    int32
playNum     int32
dtypes: int32(3)
memory usage: 330.1 MB


In [4]:
print(df.userID.max())
print(df.playerID.max())
print(df.playNum.max())
print(df.userID.min())
print(df.playerID.min())
print(df.playNum.min())

358857
160110
419157
0
0
1


## Feature Engineering

In [5]:
# pivot the matrix using sparse matrix. A low is a user, and each columns is the artist he listened, the values are the play times.
user_item_sp_mat = coo_matrix((df.playNum, (df.userID, df.playerID)), (df.userID.max()+1, df.playerID.max()+1), dtype=np.int32)

In [60]:
user_item_sp_mat

<358858x160111 sparse matrix of type '<class 'numpy.int32'>'
	with 17309316 stored elements in COOrdinate format>

# Model Selection, Comparison

#### This is a unsupervised Problem. We used Latent Dirichlet allocation(LDA) to do the dimensions reduction which is clustering. We divided 160k artists into 20 class, according to the taste of users which is the latent variable.


In [ ]:
# Run LDA model
lda = LatentDirichletAllocation(n_components=20, max_iter=10, random_state=0)
lda.fit(user_item_sp_mat)

/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


In [8]:
lda.transform(user_item_sp_mat)

array([[1.18521996e-02, 3.14321198e-01, 2.30725647e-01, ...,
        2.98017515e-02, 1.22660813e-02, 2.99096728e-06],
       [4.37706330e-02, 4.16937677e-07, 4.16937676e-07, ...,
        4.16937677e-07, 1.88498698e-01, 7.73175606e-02],
       [3.59918720e-01, 8.66400970e-06, 8.66400970e-06, ...,
        8.66400979e-06, 4.32492087e-02, 5.84304083e-01],
       ...,
       [9.24385285e-06, 4.30781116e-02, 5.99731800e-02, ...,
        1.04622041e-01, 4.14335390e-01, 3.14396062e-02],
       [6.71952695e-06, 6.71952696e-06, 4.30781335e-01, ...,
        1.66631953e-02, 6.71952695e-06, 6.71952695e-06],
       [8.48303239e-03, 8.71115999e-03, 3.36927224e-05, ...,
        3.36927225e-05, 1.91746895e-02, 3.36927231e-05]])

## Check the comparable number of points in each cluster.

In [104]:
for i in range(0,20):
    print(lda.components_[[i]].size)

160111
160111
160111
160111
160111
160111
160111
160111
160111
160111
160111
160111
160111
160111
160111
160111
160111
160111
160111
160111


In [119]:
# Check the covariance. In each cluster, inoder to check, we over it with 100000000
for i in range(0,19):
    print(np.cov(lda.components_[[i]])/100000000)

9.985488059739923
9.237519103879476
13.94716914225564
24.188243784157514
77.32077605494896
3.872186868157884
13.20738904576523
30.639151373011092
6.211481279179348
3.35539908088835
19.632366139391845
35.939598047667516
50.592676166473446
10.871454803867735
27.260207464677848
7.08892510033816
2.0340641561144874
46.088457479196805
2.9897377197721373


#### we can see that each cluster contains the same point number and in each cluster, the points are hight corrected with each other.

## Interpretations for the clusters 

In [21]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += ", ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

In [122]:
print_top_words(lda, s_player.playerName, 5)

Topic #0: tom waits, sonic youth, animal collective, pixies, the magnetic fields
Topic #1: nofx, bad religion, misfits, ramones, dropkick murphys
Topic #2: nightwish, sonata arctica, blind guardian, kamelot, apocalyptica
Topic #3: blink-182, fall out boy, my chemical romance, paramore, rise against
Topic #4: the beatles, bob dylan, the rolling stones, johnny cash, u2
Topic #5: miles davis, frank sinatra, johann sebastian bach, norah jones, amy winehouse
Topic #6: opeth, in flames, slayer, katatonia, amon amarth
Topic #7: pink floyd, metallica, iron maiden, ac/dc, queen
Topic #8: tori amos, enya, hans zimmer, enigma, yann tiersen
Topic #9: dir en grey, as i lay dying, bring me the horizon, larc~en~ciel, parkway drive
Topic #10: red hot chili peppers, tool, queens of the stone age, foo fighters, incubus
Topic #11: system of a down, linkin park, rammstein, in flames, koЯn
Topic #12: radiohead, death cab for cutie, arctic monkeys, bloc party, sufjan stevens
Topic #13: kanye west, lil wayne

#### We have 20 classes of artists now. We choose some classes to check the style of those artists. Topic 0 seems like Rock. Topic 1 is punk rock. We can conclude that if a user likes an artist in topic0, he also may like others artists in the same topic.

## Investigate properties of those clusters

In [123]:
from sklearn.externals import joblib
joblib.dump(lda, 'lda.pkl')

['lda.pkl']

In [127]:
lda.components_.shape

(20, 160111)

In [30]:
user_components = lda.transform(user_item_sp_mat)
user_components.shape

(358858, 20)

#### We reduced the dimensions of the original user-artist matrix into two matrixs. One is use 20 classes to represent users, according to the artists, the other one use 20 classes to represent artists, according to the users.

### The process to use the model. Input is a artist's name and output is the artist(s) in the same style.

In [237]:
def getName(name):
    for i in range(s_player.shape[0]):
        if s_player.iloc[i].playerName == name:
            topic=lda.components_[:, i].argmax()
            for j in range(0,9):
                artists = lda.components_[[topic]].argsort(axis=1)
                print(s_player.iloc[artists[0][-j]].playerName)

In [238]:
getName('bad religion')

back 2 bass
nofx
bad religion
misfits
ramones
dropkick murphys
rancid
against me!
the clash
